In [1]:
import pathlib
import collections
from glob import glob

import numpy as np

from astropy import units as u
from astropy.constants import c
from astropy.io import fits
from astropy import nddata, visualization
visualization.quantity_support()

from IPython import display
import ipywidgets
from ipyevents import Event 

import specutils

import tqdm

first = lambda x:next(iter(x))

%matplotlib inline
from matplotlib import style, pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg

In [2]:
cat_lines = [8498, 8542, 8662]*u.angstrom

In [3]:
datadir = pathlib.Path('/Users/erik/astrodata/DEIMOS/LeoT1')

In [4]:
ls $datadir

Allslits0.LeoT1.fits.gz
Allslits1.LeoT1.fits.gz
LeoT1.bintabs.fits
LeoT1.log
LeoT1.plan
LeoT1_1d.log
LeoT1_mcerr.log
arcwave_qa1.fits
arcwave_qa2.fits
arcwave_qa3.fits
arcwave_qa4.fits
arcwave_qa5.fits
arcwave_qa6.fits
arcwave_qa7.fits
arcwave_qa8.fits
calibSlit.LeoT1.000B.fits.gz
calibSlit.LeoT1.000R.fits.gz
calibSlit.LeoT1.001B.fits.gz
calibSlit.LeoT1.001R.fits.gz
calibSlit.LeoT1.002B.fits.gz
calibSlit.LeoT1.002R.fits.gz
calibSlit.LeoT1.003B.fits.gz
calibSlit.LeoT1.003R.fits.gz
calibSlit.LeoT1.004B.fits.gz
calibSlit.LeoT1.004R.fits.gz
calibSlit.LeoT1.005B.fits.gz
calibSlit.LeoT1.005R.fits.gz
calibSlit.LeoT1.006B.fits.gz
calibSlit.LeoT1.006R.fits.gz
calibSlit.LeoT1.007B.fits.gz
calibSlit.LeoT1.007R.fits.gz
calibSlit.LeoT1.008B.fits.gz
calibSlit.LeoT1.008R.fits.gz
calibSlit.LeoT1.009B.fits.gz
calibSlit.LeoT1.009R.fits.gz
calibSlit.LeoT1.010B.fits.gz
calibSlit.LeoT1.010R.fits.gz
calibSlit.LeoT1.011B.fits.gz
calibSlit.LeoT1.011R.fits.gz
calibSlit.LeoT1.012B.fits.gz
calibSlit.LeoT1.012R.f

In [5]:
zspecs = {}
for fn in tqdm.tqdm_notebook(glob(str(datadir / 'zspec1d.LeoT1.*.fits.gz'))):
    nm = fn.split('LeoT1.')[-1].replace('.fits.gz', '')
    d = fits.getdata(fn, 1)
    zspecs[nm] = specutils.Spectrum1D(spectral_axis=d[0]['LAMBDA']*u.angstrom, flux=d[0]['SPEC']*u.count,
                                      uncertainty=nddata.InverseVariance(d[0]['IVAR']))
    for dnm in d.dtype.names:
        if dnm not in ('SPEC', 'LAMBDA', 'IVAR'):
            zspecs[nm].meta[dnm] = d[dnm][0]

In [6]:
stemp = fits.open('templates/deimos-021507.fits')[0]
wl = 10**(stemp.header['COEFF0'] + np.arange(stemp.data.shape[1])*stemp.header['COEFF1'])*u.angstrom

template_specs = {}
for i, val in enumerate(stemp.data):
        template_specs[stemp.header[f'NAME{i}']] = specutils.Spectrum1D(spectral_axis=wl, flux=val*u.dimensionless_unscaled)

# Ipywidgets zspeccer

In [7]:
zspec_vals = collections.defaultdict(lambda:'uncategorized')
spec_comments = collections.defaultdict(lambda:'')

KEY_TO_QUALITY = {
    '0': 'uncategorized',
    '`': 'uncategorized',
    '1' : 'bad',
    '2' : 'marginal',
    '3' : 'good'
}

In [21]:
def make_fig(specname, templname, v, output, new_canvas=True):
    if new_canvas:
        fig = plt.Figure(figsize=(12, 5))
        canvas = FigureCanvasAgg(fig)
    else:
        fig = plt.figure(figsize=(12, 5))
    
    ax1, ax2 = fig.subplots(1, 2, )
    
    zline = (v*u.km/u.s/c + 1).decompose().value
    
    spec = zspecs[specname]
    if templname is not None:
        tspec = template_specs[templname]
    for ax in [ax1, ax2]:
        ax.cla()
        ax.step(spec.wavelength, spec.flux)
        data_yls = ax.get_ylim()
        if templname is None:
            ax.step([]*first(template_specs.values()).wavelength.unit, 
                    [], alpha=.5)
        else:
            ax.step(tspec.wavelength, tspec.flux.value*np.median(spec.flux))
        ax.set_ylim(*data_yls)
    ax1.set_title(specname)
    ax2.set_xlim(cat_lines[0]*zline-20*u.angstrom, cat_lines[-1]*zline+20*u.angstrom)
    
    msk2 = (ax2.get_xlim()[0]<spec.wavelength.value)&(spec.wavelength.value<ax2.get_xlim()[1])
    ax2.set_ylim(np.min(spec.flux[msk2]), np.max(spec.flux[msk2]))
    for line in cat_lines:
        ax2.axvline(line*zline, c='k', ls=':')
    
    if output is not None:
        with output:
            display.display(fig)
        
    return fig, ax1, ax2


dropdown = ipywidgets.Dropdown(options=zspecs.keys())
def update_dropdown(change):
    zspec.value = zspec_vals[dropdown.value]
    comments.value = spec_comments[dropdown.value]
dropdown.on_trait_change(update_dropdown, 'value')

left = ipywidgets.Button(description='<')
right = ipywidgets.Button(description='>')
@left.on_click
def on_left_clicked(b):
    idx = dropdown.options.index(dropdown.value)
    if idx > 0:
        dropdown.value = dropdown.options[idx-1]
@right.on_click
def on_right_clicked(b):
    idx = dropdown.options.index(dropdown.value)
    if idx < len(dropdown.options)-1:
        dropdown.value = dropdown.options[idx+1]

zspec = ipywidgets.Dropdown(options=['uncategorized', 'bad', 'marginal', 'good'], description='zspec:')
def update_zspec(change):
    zspec_vals[dropdown.value] = zspec.value
zspec.on_trait_change(update_zspec, 'value')

comments = ipywidgets.Text(description='Comments:')
def update_comments(change):
    spec_comments[dropdown.value] = comments.value
comments.on_trait_change(update_comments, 'value')

template = ipywidgets.Dropdown(description='template', options=[None]+list(template_specs.keys()))
vval = ipywidgets.FloatText(value=35, description='v (km/s)')

    
top = ipywidgets.HBox([left, dropdown, right])
middle = ipywidgets.HBox([zspec, comments])
lower = ipywidgets.HBox([template, vval])

ui = ipywidgets.VBox([top, middle, lower])

out = ipywidgets.Output()
 
d = Event(source=out, watched_events=['keydown'])
def handle_event(event):
    try:
        if event['key'] in KEY_TO_QUALITY.keys():
            zspec.value = KEY_TO_QUALITY[event['key']]
        elif event['key'] in ('-', '[',','):
            left.click()
        elif event['key'] in ('+', ']','.'):
            right.click()
    except Exception as e:
        comments.value = repr(e)
d.on_dom_event(handle_event)

display.display(ui, out)


fig, ax1, ax2 = make_fig(dropdown.value, template.value, vval.value, out)
def update_fig(change):
    specname = dropdown.value
    templname = template.value
    v = vval.value
    
    zline = (v*u.km/u.s/c + 1).decompose().value
    spec = zspecs[specname]
    if templname is not None:
        tspec = template_specs[templname]
        
    for ax in [ax1, ax2]:
        ax.get_children()[0].set_data(spec.wavelength, spec.flux)
        if templname is not None:
            ax.get_children()[1].set_data(tspec.wavelength, tspec.flux.value*np.median(spec.flux))
        ax.set_ylim(np.min(spec.flux), np.max(spec.flux))
    
    ax1.set_title(specname)
    ax2.set_xlim(cat_lines[0]*zline-20*u.angstrom, cat_lines[-1]*zline+20*u.angstrom)
    
    msk2 = (ax2.get_xlim()[0]<spec.wavelength.value)&(spec.wavelength.value<ax2.get_xlim()[1])
    ax2.set_ylim(np.min(spec.flux[msk2]), np.max(spec.flux[msk2]))
    
    with out:
        display.clear_output(True)
        display.display(fig)
    
for widget in (dropdown, template, vval):
    widget.on_trait_change(update_fig, 'value')

/Users/erik/miniconda3/envs/astro36/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: on_trait_change is deprecated in traitlets 4.1: use observe instead
/Users/erik/miniconda3/envs/astro36/lib/python3.6/site-packages/ipykernel_launcher.py:62: DeprecationWarning: on_trait_change is deprecated in traitlets 4.1: use observe instead
/Users/erik/miniconda3/envs/astro36/lib/python3.6/site-packages/ipykernel_launcher.py:67: DeprecationWarning: on_trait_change is deprecated in traitlets 4.1: use observe instead


Output()

/Users/erik/miniconda3/envs/astro36/lib/python3.6/site-packages/ipykernel_launcher.py:125: DeprecationWarning: on_trait_change is deprecated in traitlets 4.1: use observe instead


In [9]:
zspec_vals, spec_comments

(defaultdict(<function __main__.<lambda>()>, {}),
 defaultdict(<function __main__.<lambda>()>, {}))

Marla requests:

* Fast enough to go fast
* hit number buttons for zspeccing (ipyevents)
* 2D slit

* (some quality plots/chi^2 surfaces - understandable interface to Marla)